In [1]:
# https://catastotn.tndigit.it/scarico-catasto-geometrico/it/index.html
import geopandas as gpd
import pandas as pd
import os 
import glob
import requests
import zipfile
dest_doc = ".." + os.sep + "docs"
donwload_url = "https://catastotn.tndigit.it/export_semestrale_VL_PUBB/IDR0020230701_TIPOCATSH_CCXXX.zip"
donwload_url = "https://catastotn.tndigit.it/export_semestrale_VL_PUBB/IDR0020240101_TIPOCATSH_CCXXX.zip"
url_csv = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSPeLuWTTF1JhWOhhR_ZJmSLBJhMqcJ771xWUeNnuX2co7aV2k2UytMRWU3AZdgfP4gIsWZZHsmx3T7/pub?output=csv"
src_comunicatastaliamministrativi = "comuni_catastali_amministrativi_trentino.csv"
comunicatastaliamministrativi = pd.read_csv(src_comunicatastaliamministrativi)

In [2]:
def getComuneAmministrativo(name):
    if name == "FOLAS":
        name = 'FOLAS-REVIAN'
    if name =="BORGHETTO A/A":
        name = "BORGHETTO"
    amministrativo = ""
    rname = comunicatastaliamministrativi[comunicatastaliamministrativi['ComuneCatastale'].str.upper() == name.upper()]["Comune Amministartivo"]
    if len(rname) >0:
        amministrativo = rname.values[0]
    else: 
        print(name)
    return (amministrativo)

In [3]:
df = pd.read_csv(url_csv)
codici_catastali = df.codice_comune_catastale.unique()

In [4]:
gdflist = []
for codice in codici_catastali:
    codice = str(codice).zfill(3).replace(".0","")
    url = donwload_url.replace("XXX",codice)
    response = requests.get(url)
    # Elenca i nomi dei file che vuoi estrarre
    files_to_extract = []
    suffix = "_vl_uniqueparcel_poly"
    suffix = "_vl_parcel_poly"
    files_to_extract.append(codice + suffix + ".prj")
    files_to_extract.append(codice + suffix + ".shp")
    files_to_extract.append(codice + suffix + ".shx")
    files_to_extract.append(codice + suffix + ".dbf")
    if response.status_code == 200:
        with open("file.zip", "wb") as file:
            file.write(response.content)
        with zipfile.ZipFile("file.zip", "r") as zip_ref:
            for file_name in zip_ref.namelist():
                if file_name in files_to_extract:
                    zip_ref.extract(file_name)
        gdf = gpd.read_file(codice + suffix + ".shp")
        crs = gdf.crs
        gdflist.append(gdf)
        os.remove("file.zip")
        for shp in files_to_extract:
            os.remove(shp)
parcels = gpd.GeoDataFrame(pd.concat(gdflist, ignore_index=True), crs=crs)

In [6]:
parcels['catasto'] = ""
parcels['comune'] = "NO"
parcels['ufficio'] = ""
parcels['prg1'] = ""
parcels['prg2'] = ""
parcels['prg3'] = ""
parcels['prg4'] = ""
parcels['prg5'] = ""
parcels['aggiornamento'] = ""
parcels["ettari"] = ""
notfound = []
for idx, row in df.iterrows():
    codice_comune_catastale= row['codice_comune_catastale']
    codice_particella = row['codice_particella']  
    p = parcels[(parcels.PT_CODE == codice_particella) & (parcels.PT_CCAT == codice_comune_catastale)]  
    if p.shape[0] >0:
        parcels.at[p.index[0],"ufficio"] = row['comune_ammistrativo']
        parcels.at[p.index[0],"comune"] = getComuneAmministrativo(row['nome_comune_catastale'])
        parcels.at[p.index[0],"catasto"] = row['nome_comune_catastale']
        parcels.at[p.index[0],"prg1"] = row['destinazione_uso_1']
        parcels.at[p.index[0],"prg2"] = row['destinazione_uso_2']
        parcels.at[p.index[0],"prg3"] = row['commento']
        #parcels.at[p.index[0],"prg4"] = row['destinazione_uso_3']
        #parcels.at[p.index[0],"prg5"] = row['destinazione_uso_4']
        #parcels.at[p.index[0],"prg6"] = row['destinazione_uso_5']
        parcels.at[p.index[0],"aggiornamento"] = row['data_ultimo_aggiornamento_dati']
        parcels.at[p.index[0],"ettari"] = round(p.geometry.area[p.index[0]]/1000,2)
    else:
        nf = {}
        nf['codice_particella'] = codice_particella
        nf['codice_comune_catastale'] = codice_comune_catastale
        notfound.append(nf)
        

.1691
101
104
125
167
211
277
234
243
244
246
247
248
249
256
257
258
276
277/2
278
280
394
436
440
462
522
540
689
690
730
731/2
803/1
842
1018/2
1172
1313
1376/1
1376/2
1430/1
1430/2
1431
1465/2
1896/1
1897
1901/1
1901/2
2036
2158/3
2159
2259
2260
2285
2286
2287
2288
2298
2319/2
2320/1
2341
2406
2414
2591
2593
2613
2614
2623
2625
2638
2639/1
2639/3
2639/4
2639/5
2639/15
2634/3
1688/2
1688/3
3982
3983
218
885/1
885/3
886/1
.295
96
97
98
99
130
197/4
197/5
232
233
268
281
298
319
1155/2
1218
1219
1248
1249
1250
1273/1
1394/2
1421
1422
1450
1451/1
1453/1
1453/2
378
810
901
916/2
916/3
925
996/1
966/2
997/1
998
1040/2
1040/3
1040/47
1040/48
1044/2
1127
1040/39
1042
44
58/1
58/2
115/1
115/4
124
197
199
307
349
518/1
519
1040
413
506
824
888
889
890
891
897
898
899
900
901
101/2
104/1
36
102
103
614
2168
2332
2335
2365
3012
3053
1084
1103/1
1104
1118
1119
1120/1
1120/3
2171/1
2219
2221
2222
2223
2224
2225
2226
2227
2239
2240
2241
2242/1
2259/5
2262/1
2262/6
2268
2269
2277/1
2279/1
2282/1
2

In [7]:
parcels.fillna("non disponibile", inplace=True)

/tmp/ipykernel_7058/1536920151.py:1: DeprecationWarning: ExtensionArray.fillna added a 'copy' keyword in pandas 2.1.0. In a future version, ExtensionArray subclasses will need to implement this keyword or an exception will be raised. In the interim, the keyword is ignored by GeometryArray.
  parcels.fillna("non disponibile", inplace=True)


In [8]:
usi_civici = parcels[parcels.comune != "NO"]
usi_civici=usi_civici.to_crs(epsg=4326)
usi_civici.to_file(dest_doc + os.sep + "usi_civici.geojson")

In [9]:
usi_civici_edifici = usi_civici[usi_civici['PT_CODE'].str.startswith('.')]
usi_civici_terreni = usi_civici[~usi_civici['PT_CODE'].str.startswith('.')]

In [10]:
pd.DataFrame(notfound).to_excel(dest_doc + os.sep + "particelle_non_trovate.xlsx")
usi_civici_terreni.to_file(dest_doc + os.sep +"usi_civici_terreni.geojson")
usi_civici_edifici.to_file(dest_doc + os.sep +"usi_civici_edifici.geojson")